Meta-Learning for Few-Shot Log Anomaly Detection in Extreme Imbalance Settings

In [1]:
import os
import sys
import pickle
import warnings
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from collections import defaultdict, Counter
import gc
from concurrent.futures import ThreadPoolExecutor, as_completed
import threading

warnings.filterwarnings('ignore')

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.optim import Adam, SGD
import torch.multiprocessing as mp

from sklearn.metrics import f1_score, accuracy_score, balanced_accuracy_score, matthews_corrcoef
from sklearn.metrics import precision_score, recall_score, roc_auc_score, average_precision_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

In [2]:
SEED = 42
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

ROOT = Path(r"C:\Computer Science\AIMLDL\log-anomaly-detection")
FEAT_PATH = ROOT / "features"
RESULTS_PATH = ROOT / "results" / "meta_learning"
MODELS_PATH = ROOT / "models" / "meta_learning"

RESULTS_PATH.mkdir(parents=True, exist_ok=True)
MODELS_PATH.mkdir(parents=True, exist_ok=True)

Using device: cuda
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [3]:
feat_file = FEAT_PATH / "enhanced_imbalanced_features.pkl"
with open(feat_file, 'rb') as f:
    feat_data = pickle.load(f)
    data_dict = feat_data['hybrid_features_data']
    num_classes = feat_data['config'].get('num_classes', 2)

split_file = FEAT_PATH / "enhanced_cross_source_splits.pkl"
with open(split_file, 'rb') as f:
    split_data = pickle.load(f)
    splits = split_data['splits']

print(f"Loaded {len(data_dict)} sources")
print(f"Classes: {num_classes}")

LABEL_MAP = {0: 'normal', 1: 'anomaly'}

Loaded 16 sources
Classes: 2


In [4]:
def calculate_metrics(y_true, y_pred, y_proba=None):
    metrics = {}
    metrics['accuracy'] = accuracy_score(y_true, y_pred)
    metrics['balanced_acc'] = balanced_accuracy_score(y_true, y_pred)
    metrics['f1_macro'] = f1_score(y_true, y_pred, average='macro', zero_division=0)
    metrics['f1_weighted'] = f1_score(y_true, y_pred, average='weighted', zero_division=0)
    metrics['mcc'] = matthews_corrcoef(y_true, y_pred)
    
    per_class = {}
    for cls in np.unique(np.concatenate([y_true, y_pred])):
        y_true_bin = (y_true == cls).astype(int)
        y_pred_bin = (y_pred == cls).astype(int)
        if y_true_bin.sum() > 0:
            per_class[int(cls)] = {
                'precision': precision_score(y_true_bin, y_pred_bin, zero_division=0),
                'recall': recall_score(y_true_bin, y_pred_bin, zero_division=0),
                'f1': f1_score(y_true_bin, y_pred_bin, zero_division=0),
                'support': int(y_true_bin.sum())
            }
    metrics['per_class'] = per_class
    
    if y_proba is not None and len(np.unique(y_true)) == 2:
        try:
            metrics['auroc'] = roc_auc_score(y_true, y_proba[:, 1])
            metrics['auprc'] = average_precision_score(y_true, y_proba[:, 1])
        except:
            metrics['auroc'] = 0.0
            metrics['auprc'] = 0.0
    else:
        metrics['auroc'] = 0.0
        metrics['auprc'] = 0.0
    
    return metrics


def create_few_shot_episode(X, y, n_way, k_shot, q_query, balance=True):
    classes = np.unique(y)
    if len(classes) < n_way:
        return None, None, None, None
    
    selected_classes = np.random.choice(classes, n_way, replace=False)
    
    support_X, support_y = [], []
    query_X, query_y = [], []
    
    for cls in selected_classes:
        cls_indices = np.where(y == cls)[0]
        
        if len(cls_indices) < k_shot + q_query:
            if balance:
                return None, None, None, None
            else:
                available = len(cls_indices)
                k_use = min(k_shot, available // 2)
                q_use = available - k_use
                if k_use == 0 or q_use == 0:
                    return None, None, None, None
        else:
            k_use = k_shot
            q_use = q_query
        
        selected = np.random.choice(cls_indices, k_use + q_use, replace=False)
        support_indices = selected[:k_use]
        query_indices = selected[k_use:k_use + q_use]
        
        support_X.append(X[support_indices])
        support_y.append(y[support_indices])
        query_X.append(X[query_indices])
        query_y.append(y[query_indices])
    
    support_X = np.vstack(support_X)
    support_y = np.concatenate(support_y)
    query_X = np.vstack(query_X)
    query_y = np.concatenate(query_y)
    
    shuffle_support = np.random.permutation(len(support_y))
    support_X = support_X[shuffle_support]
    support_y = support_y[shuffle_support]
    
    shuffle_query = np.random.permutation(len(query_y))
    query_X = query_X[shuffle_query]
    query_y = query_y[shuffle_query]
    
    return support_X, support_y, query_X, query_y

def create_imbalanced_episode(X, y, minority_k_shot, majority_k_shot, q_query_per_class):
    classes = np.unique(y)
    if len(classes) != 2:
        return None, None, None, None
    
    class_counts = [np.sum(y == cls) for cls in classes]
    minority_cls = classes[np.argmin(class_counts)]
    majority_cls = classes[np.argmax(class_counts)]
    
    minority_indices = np.where(y == minority_cls)[0]
    majority_indices = np.where(y == majority_cls)[0]
    
    if len(minority_indices) < minority_k_shot + q_query_per_class:
        return None, None, None, None
    if len(majority_indices) < majority_k_shot + q_query_per_class:
        return None, None, None, None
    
    minority_selected = np.random.choice(minority_indices, minority_k_shot + q_query_per_class, replace=False)
    majority_selected = np.random.choice(majority_indices, majority_k_shot + q_query_per_class, replace=False)
    
    support_X = np.vstack([X[minority_selected[:minority_k_shot]], X[majority_selected[:majority_k_shot]]])
    support_y = np.concatenate([y[minority_selected[:minority_k_shot]], y[majority_selected[:majority_k_shot]]])
    
    query_X = np.vstack([X[minority_selected[minority_k_shot:]], X[majority_selected[majority_k_shot:]]])
    query_y = np.concatenate([y[minority_selected[minority_k_shot:]], y[majority_selected[majority_k_shot:]]])
    
    shuffle_support = np.random.permutation(len(support_y))
    support_X = support_X[shuffle_support]
    support_y = support_y[shuffle_support]
    
    shuffle_query = np.random.permutation(len(query_y))
    query_X = query_X[shuffle_query]
    query_y = query_y[shuffle_query]
    
    return support_X, support_y, query_X, query_y


def prototypical_loss(embeddings, labels, n_way):
    classes = torch.unique(labels)
    prototypes = []
    
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = embeddings[cls_mask]
        prototype = cls_embeddings.mean(dim=0)
        prototypes.append(prototype)
    
    prototypes = torch.stack(prototypes)
    
    distances = torch.cdist(embeddings, prototypes, p=2)
    log_probs = F.log_softmax(-distances, dim=1)
    
    loss = F.nll_loss(log_probs, labels)
    return loss

def focal_loss(logits, labels, alpha=0.25, gamma=2.0):
    ce_loss = F.cross_entropy(logits, labels, reduction='none')
    pt = torch.exp(-ce_loss)
    focal_loss = alpha * (1 - pt) ** gamma * ce_loss
    return focal_loss.mean()

def contrastive_loss(embeddings, labels, temperature=0.5):
    embeddings = F.normalize(embeddings, dim=1)
    similarity_matrix = torch.matmul(embeddings, embeddings.T) / temperature
    
    labels = labels.contiguous().view(-1, 1)
    mask = torch.eq(labels, labels.T).float().to(device)
    
    logits_max, _ = torch.max(similarity_matrix, dim=1, keepdim=True)
    logits = similarity_matrix - logits_max.detach()
    
    exp_logits = torch.exp(logits)
    log_prob = logits - torch.log(exp_logits.sum(1, keepdim=True))
    
    mask_sum = mask.sum(1)
    mask_sum = torch.clamp(mask_sum, min=1.0)
    
    mean_log_prob_pos = (mask * log_prob).sum(1) / mask_sum
    loss = -mean_log_prob_pos.mean()
    
    return loss


def meta_network(input_dim, hidden_dims=[256, 128], output_dim=64, dropout=0.3):
    layers = []
    prev_dim = input_dim
    for hidden_dim in hidden_dims:
        layers.append(nn.Linear(prev_dim, hidden_dim))
        layers.append(nn.BatchNorm1d(hidden_dim))
        layers.append(nn.ReLU())
        layers.append(nn.Dropout(dropout))
        prev_dim = hidden_dim
    layers.append(nn.Linear(prev_dim, output_dim))
    return nn.Sequential(*layers)

def classifier_head(input_dim, num_classes=2, dropout=0.2):
    return nn.Sequential(
        nn.Linear(input_dim, input_dim // 2),
        nn.ReLU(),
        nn.Dropout(dropout),
        nn.Linear(input_dim // 2, num_classes)
    )

def attention_pooling(input_dim):
    return nn.Sequential(
        nn.Linear(input_dim, 1),
        nn.Softmax(dim=1)
    )

def maml_inner_loop(model, support_X, support_y, inner_lr, inner_steps, loss_fn):
    model_copy = MetaLearner(model.input_dim, model.hidden_dims, model.embedding_dim, 
                             model.dropout, model.num_classes).to(device)
    model_copy.load_state_dict(model.state_dict())
    
    optimizer = SGD(model_copy.parameters(), lr=inner_lr)
    
    support_X_tensor = torch.FloatTensor(support_X).to(device)
    support_y_tensor = torch.LongTensor(support_y).to(device)
    
    for step in range(inner_steps):
        optimizer.zero_grad()
        logits = model_copy.predict(support_X_tensor)
        loss = loss_fn(logits, support_y_tensor)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model_copy.parameters(), 1.0)
        optimizer.step()
    
    return model_copy

def memory_efficient_maml(model, support_X, support_y, inner_lr, inner_steps, loss_fn):
    params = {name: param.clone() for name, param in model.named_parameters()}
    
    support_X_tensor = torch.FloatTensor(support_X).to(device)
    support_y_tensor = torch.LongTensor(support_y).to(device)
    
    for step in range(inner_steps):
        embeddings = model(support_X_tensor)
        logits = model.classifier(embeddings)
        loss = loss_fn(logits, support_y_tensor)
        
        grads = torch.autograd.grad(loss, model.parameters(), create_graph=True)
        
        for (name, param), grad in zip(model.named_parameters(), grads):
            param.data = param.data - inner_lr * grad
    
    result_params = {name: param.clone() for name, param in model.named_parameters()}
    
    for name, param in model.named_parameters():
        param.data = params[name].data
    
    return result_params

def process_single_task(encoder_state, classifier_state, source_name, source_features, source_labels, 
                       source_k_shots, inner_lr, inner_steps, q_query, device_id, input_dim, hidden_dims, 
                       embedding_dim, dropout, num_classes):
    local_device = torch.device(f"cuda:{device_id}" if torch.cuda.is_available() else "cpu")
    
    local_encoder = meta_network(input_dim, hidden_dims, embedding_dim, dropout).to(local_device)
    local_encoder.load_state_dict(encoder_state)
    local_encoder.input_dim = input_dim
    local_encoder.hidden_dims = hidden_dims
    local_encoder.output_dim = embedding_dim
    local_encoder.dropout = dropout
    
    local_classifier = classifier_head(embedding_dim, num_classes, dropout).to(local_device)
    local_classifier.load_state_dict(classifier_state)
    local_encoder.classifier = local_classifier
    
    X_source = source_features[source_name]
    y_source = source_labels[source_name]
    k_shots = source_k_shots[source_name]
    
    episode = create_imbalanced_episode(
        X_source, y_source, 
        k_shots['minority'], k_shots['majority'], q_query
    )
    
    if episode[0] is None:
        return None
    
    support_X, support_y, query_X, query_y = episode
    
    adapted_model = maml_inner_loop(
        local_encoder, support_X, support_y, 
        inner_lr, inner_steps, focal_loss
    )
    
    query_X_tensor = torch.FloatTensor(query_X).to(local_device)
    query_y_tensor = torch.LongTensor(query_y).to(local_device)
    
    query_embeddings = adapted_model(query_X_tensor)
    query_logits = adapted_model.classifier(query_embeddings)
    
    task_loss = focal_loss(query_logits, query_y_tensor)
    
    return task_loss.cpu()

def compute_prototypes(embeddings, labels):
    classes = torch.unique(labels)
    prototypes = []
    for cls in classes:
        cls_mask = labels == cls
        cls_embeddings = embeddings[cls_mask]
        prototype = cls_embeddings.mean(dim=0)
        prototypes.append(prototype)
    return torch.stack(prototypes), classes

In [5]:
input_dim = 200
hidden_dims = [256, 128]
embedding_dim = 64
dropout = 0.3

class MetaLearner(nn.Module):
    def __init__(self, input_dim, hidden_dims, embedding_dim, dropout, num_classes):
        super(MetaLearner, self).__init__()
        self.input_dim = input_dim
        self.hidden_dims = hidden_dims
        self.embedding_dim = embedding_dim
        self.dropout = dropout
        self.num_classes = num_classes
        
        self.encoder = meta_network(input_dim, hidden_dims, embedding_dim, dropout)
        self.classifier = classifier_head(embedding_dim, num_classes, dropout)
    
    def forward(self, x):
        embeddings = self.encoder(x)
        return embeddings
    
    def predict(self, x):
        embeddings = self.encoder(x)
        logits = self.classifier(embeddings)
        return logits

In [6]:
model = MetaLearner(input_dim, hidden_dims, embedding_dim, dropout, num_classes).to(device)

print(f"Model parameters: {sum(p.numel() for p in model.parameters()):,}")
print(f"Encoder parameters: {sum(p.numel() for p in model.encoder.parameters()):,}")
print(f"Classifier parameters: {sum(p.numel() for p in model.classifier.parameters()):,}")

Model parameters: 95,522
Encoder parameters: 93,376
Classifier parameters: 2,146


In [7]:
QUICK_TEST = False

if QUICK_TEST:
    meta_lr = 1e-3
    inner_lr = 1e-2
    inner_steps = 3
    meta_batch_size = 2
    num_meta_iterations = 50
    k_shot_minority = 3
    k_shot_majority = 5
    q_query = 10
    early_stopping_patience = 20
    min_delta = 1e-4
    print("\nQUICK TEST MODE ENABLED")
else:
    meta_lr = 1e-3
    inner_lr = 1e-2
    inner_steps = 5
    meta_batch_size = 8
    num_meta_iterations = 1000
    k_shot_minority = 5
    k_shot_majority = 10
    q_query = 15
    early_stopping_patience = 100
    min_delta = 1e-4

meta_optimizer = Adam(model.parameters(), lr=meta_lr)

from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts, ReduceLROnPlateau
scheduler = CosineAnnealingWarmRestarts(meta_optimizer, T_0=100, T_mult=2, eta_min=1e-5)

print(f"\nMeta-Learning Configuration:")
print(f"  Meta LR: {meta_lr}")
print(f"  Inner LR: {inner_lr}")
print(f"  Inner steps: {inner_steps}")
print(f"  Meta batch size: {meta_batch_size}")
print(f"  Iterations: {num_meta_iterations}")
print(f"  K-shot (minority): {k_shot_minority}")
print(f"  K-shot (majority): {k_shot_majority}")
print(f"  Query samples: {q_query}")
print(f"  Early stopping patience: {early_stopping_patience}")
print(f"  LR scheduler: CosineAnnealingWarmRestarts")


Meta-Learning Configuration:
  Meta LR: 0.001
  Inner LR: 0.01
  Inner steps: 5
  Meta batch size: 8
  Iterations: 1000
  K-shot (minority): 5
  K-shot (majority): 10
  Query samples: 15
  Early stopping patience: 100
  LR scheduler: CosineAnnealingWarmRestarts


In [8]:
train_sources = []
source_imbalance_ratios = {}

for source_name, source_data in data_dict.items():
    if source_data['labels'] is not None:
        labels = source_data['labels']
        if len(np.unique(labels)) >= 2:
            train_sources.append(source_name)
            unique, counts = np.unique(labels, return_counts=True)
            imb_ratio = counts.max() / counts.min() if len(counts) > 1 else 1.0
            source_imbalance_ratios[source_name] = imb_ratio

train_sources_sorted = sorted(train_sources, key=lambda x: source_imbalance_ratios[x])

print(f"\nTraining sources: {len(train_sources)}")
print(f"Sources (sorted by imbalance): {train_sources_sorted}")
print(f"Imbalance ratios: {[(s, f'{source_imbalance_ratios[s]:.1f}:1') for s in train_sources_sorted[:5]]}")


Training sources: 13
Sources (sorted by imbalance): ['HPC_2k', 'Windows_2k', 'Hadoop_2k', 'Apache_2k', 'Zookeeper_2k', 'Mac_2k', 'Thunderbird_2k', 'BGL_2k', 'Proxifier_2k', 'Linux_2k', 'Android_2k', 'HealthApp_2k', 'Spark_2k']
Imbalance ratios: [('HPC_2k', '1.2:1'), ('Windows_2k', '1.5:1'), ('Hadoop_2k', '1.9:1'), ('Apache_2k', '2.5:1'), ('Zookeeper_2k', '2.9:1')]


In [9]:
feat_variant = 'selected_imbalanced'

source_features = {}
source_labels = {}
source_scalers = {}
source_k_shots = {}

for source_name in train_sources:
    source_data = data_dict[source_name]
    if feat_variant in source_data['feature_variants']:
        X = source_data['feature_variants'][feat_variant]
        y = source_data['labels']
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(X)
        
        source_features[source_name] = X_scaled
        source_labels[source_name] = y
        source_scalers[source_name] = scaler
        
        unique, counts = np.unique(y, return_counts=True)
        imb_ratio = counts.max() / counts.min() if len(counts) > 1 else 1.0
        minority_count = counts.min()
        
        k_min = max(2, min(k_shot_minority, minority_count // 4))
        k_maj = max(k_min, min(k_shot_majority, minority_count // 2))
        source_k_shots[source_name] = {'minority': k_min, 'majority': k_maj}
        
        print(f"  {source_name}: {len(y)} samples, imbalance {imb_ratio:.2f}:1, k-shot {k_min}/{k_maj}")

print(f"\nPrepared {len(source_features)} sources for meta-training")

  Android_2k: 2000 samples, imbalance 75.92:1, k-shot 5/10
  Apache_2k: 2000 samples, imbalance 2.50:1, k-shot 5/10
  BGL_2k: 2000 samples, imbalance 10.90:1, k-shot 5/10
  Hadoop_2k: 2000 samples, imbalance 1.90:1, k-shot 5/10
  HealthApp_2k: 2000 samples, imbalance 180.82:1, k-shot 2/5
  HPC_2k: 2000 samples, imbalance 1.25:1, k-shot 5/10
  Linux_2k: 2000 samples, imbalance 20.28:1, k-shot 5/10
  Mac_2k: 2000 samples, imbalance 3.62:1, k-shot 5/10
  Proxifier_2k: 2000 samples, imbalance 19.62:1, k-shot 5/10
  Spark_2k: 2000 samples, imbalance 249.00:1, k-shot 2/4
  Thunderbird_2k: 2000 samples, imbalance 9.26:1, k-shot 5/10
  Windows_2k: 2000 samples, imbalance 1.53:1, k-shot 5/10
  Zookeeper_2k: 2000 samples, imbalance 2.89:1, k-shot 5/10

Prepared 13 sources for meta-training


In [10]:
print("\nStarting Meta-Training with improvements...")

best_meta_loss = float('inf')
meta_losses = []
patience_counter = 0
curriculum_phase = 0
curriculum_thresholds = [5, 20, 100]

def get_curriculum_sources(phase, sources_sorted, imbalance_ratios):
    if phase == 0:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[0]]
    elif phase == 1:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[1]]
    elif phase == 2:
        return [s for s in sources_sorted if imbalance_ratios[s] <= curriculum_thresholds[2]]
    else:
        return sources_sorted

curriculum_sources = get_curriculum_sources(curriculum_phase, train_sources_sorted, source_imbalance_ratios)
if not curriculum_sources:
    curriculum_sources = train_sources_sorted[:len(train_sources_sorted)//2]


Starting Meta-Training with improvements...


In [11]:
print(f"Starting with curriculum phase {curriculum_phase}: {len(curriculum_sources)} sources")

for iteration in range(num_meta_iterations):
    if iteration > 0 and iteration % 200 == 0 and curriculum_phase < 3:
        curriculum_phase += 1
        curriculum_sources = get_curriculum_sources(curriculum_phase, train_sources_sorted, source_imbalance_ratios)
        if not curriculum_sources:
            curriculum_sources = train_sources_sorted
        print(f"\nCurriculum phase {curriculum_phase}: {len(curriculum_sources)} sources")
    
    meta_optimizer.zero_grad()
    meta_loss = 0.0
    valid_tasks = 0
    
    task_losses = []
    
    for batch_idx in range(meta_batch_size):
        available_sources = [s for s in curriculum_sources if s in source_features]
        if not available_sources:
            available_sources = list(source_features.keys())
        
        source_name = np.random.choice(available_sources)
        X_source = source_features[source_name]
        y_source = source_labels[source_name]
        k_shots = source_k_shots[source_name]
        
        episode = create_imbalanced_episode(
            X_source, y_source, 
            k_shots['minority'], k_shots['majority'], q_query
        )
        
        if episode[0] is None:
            continue
        
        support_X, support_y, query_X, query_y = episode
        
        adapted_model = maml_inner_loop(
            model, support_X, support_y, 
            inner_lr, inner_steps, focal_loss
        )
        
        query_X_tensor = torch.FloatTensor(query_X).to(device)
        query_y_tensor = torch.LongTensor(query_y).to(device)
        
        query_logits = adapted_model.predict(query_X_tensor)
        
        task_loss = focal_loss(query_logits, query_y_tensor)
        task_losses.append(task_loss)
        valid_tasks += 1
    
    if valid_tasks > 0:
        meta_loss = torch.stack(task_losses).mean()
        meta_loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        meta_optimizer.step()
        scheduler.step()
        
        current_loss = meta_loss.item()
        meta_losses.append(current_loss)
        
        if current_loss < best_meta_loss - min_delta:
            best_meta_loss = current_loss
            patience_counter = 0
            torch.save({
                'model': model.state_dict(),
                'iteration': iteration,
                'meta_loss': best_meta_loss,
                'curriculum_phase': curriculum_phase
            }, MODELS_PATH / 'best_meta_model.pt')
        else:
            patience_counter += 1
        
        if (iteration + 1) % 50 == 0:
            avg_loss = np.mean(meta_losses[-50:])
            current_lr = meta_optimizer.param_groups[0]['lr']
            print(f"Iter {iteration + 1}/{num_meta_iterations} - Loss: {avg_loss:.4f}, LR: {current_lr:.6f}, Patience: {patience_counter}/{early_stopping_patience}")
        
        if patience_counter >= early_stopping_patience:
            print(f"\nEarly stopping at iteration {iteration + 1}")
            break

print(f"\nMeta-training complete. Best meta loss: {best_meta_loss:.4f}")

Starting with curriculum phase 0: 6 sources
Iter 50/1000 - Loss: 0.0459, LR: 0.000505, Patience: 43/100
Iter 100/1000 - Loss: 0.0456, LR: 0.001000, Patience: 42/100
Iter 150/1000 - Loss: 0.0458, LR: 0.000855, Patience: 25/100
Iter 200/1000 - Loss: 0.0457, LR: 0.000505, Patience: 75/100

Curriculum phase 1: 9 sources

Early stopping at iteration 225

Meta-training complete. Best meta loss: 0.0417


In [12]:
checkpoint = torch.load(MODELS_PATH / 'best_meta_model.pt')
model.load_state_dict(checkpoint['model'])
print(f"Loaded best meta-model from iteration {checkpoint['iteration']}")

print("\nEvaluating on test sources...")

test_results = []

test_splits = splits[:3] if QUICK_TEST else splits

for split in test_splits:
    test_source = split['test_source']
    train_sources_split = split['train_sources']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None:
        continue
    
    if feat_variant not in test_data['feature_variants']:
        continue
    
    X_test = test_data['feature_variants'][feat_variant]
    y_test = test_data['labels']
    
    if len(np.unique(y_test)) < 2:
        continue
    
    scaler_test = StandardScaler()
    X_test_scaled = scaler_test.fit_transform(X_test)
    
    X_train_list = []
    y_train_list = []
    
    for src in train_sources_split:
        if src in source_features:
            X_train_list.append(source_features[src])
            y_train_list.append(source_labels[src])
    
    if not X_train_list:
        continue
    
    X_train_combined = np.vstack(X_train_list)
    y_train_combined = np.concatenate(y_train_list)
    
    unique, counts = np.unique(y_train_combined, return_counts=True)
    minority_count = counts.min()
    
    k_shot_adapt = min(k_shot_minority, minority_count // 2)
    if k_shot_adapt < 2:
        continue
    
    episode = create_imbalanced_episode(
        X_train_combined, y_train_combined,
        k_shot_adapt, k_shot_adapt * 2, 10
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    adapted_model = maml_inner_loop(
        model, support_X, support_y,
        inner_lr, inner_steps * 2, focal_loss
    )
    
    adapted_model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        test_logits = adapted_model.predict(X_test_tensor)
        test_probs = F.softmax(test_logits, dim=1).cpu().numpy()
        test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test, test_preds, test_probs)
    
    test_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'test_samples': len(y_test),
        'support_samples': len(support_y)
    })
    
    print(f"\n{test_source}:")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")
    print(f"  MCC: {metrics['mcc']:.4f}")

Loaded best meta-model from iteration 124

Evaluating on test sources...

Android_2k:
  F1-Macro: 0.3383
  Balanced Acc: 0.7300
  AUROC: 0.8929
  MCC: 0.1047

Apache_2k:
  F1-Macro: 0.2195
  Balanced Acc: 0.4760
  AUROC: 0.8086
  MCC: -0.1548

BGL_2k:
  F1-Macro: 0.1689
  Balanced Acc: 0.2253
  AUROC: 0.1926
  MCC: -0.3661

Hadoop_2k:
  F1-Macro: 0.3050
  Balanced Acc: 0.3173
  AUROC: 0.2970
  MCC: -0.3875

HealthApp_2k:
  F1-Macro: 0.1413
  Balanced Acc: 0.5779
  AUROC: 0.6447
  MCC: 0.0319

HPC_2k:
  F1-Macro: 0.2531
  Balanced Acc: 0.3129
  AUROC: 0.3385
  MCC: -0.4386

Linux_2k:
  F1-Macro: 0.4578
  Balanced Acc: 0.4741
  AUROC: 0.4666
  MCC: -0.0264

Mac_2k:
  F1-Macro: 0.3800
  Balanced Acc: 0.5642
  AUROC: 0.5456
  MCC: 0.1296

Proxifier_2k:
  F1-Macro: 0.2243
  Balanced Acc: 0.2786
  AUROC: 0.2376
  MCC: -0.2096

Spark_2k:
  F1-Macro: 0.1537
  Balanced Acc: 0.4634
  AUROC: 0.4470
  MCC: -0.0121

Thunderbird_2k:
  F1-Macro: 0.0973
  Balanced Acc: 0.5016
  AUROC: 0.8886
  MCC: 0.

In [13]:
if test_results:
    df_results = pd.DataFrame(test_results)
    df_results = df_results.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("META-LEARNING EVALUATION SUMMARY")
    print("="*80)
    print(df_results.to_string(index=False))
    
    print("\n" + "="*60)
    print("AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(test_results)}")
    print(f"Average F1-Macro: {df_results['f1_macro'].mean():.4f} ± {df_results['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_results['balanced_acc'].mean():.4f} ± {df_results['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_results['auroc'].mean():.4f} ± {df_results['auroc'].std():.4f}")
    print(f"Average MCC: {df_results['mcc'].mean():.4f} ± {df_results['mcc'].std():.4f}")
    
    results_file = RESULTS_PATH / f"meta_learning_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_results.to_csv(results_file, index=False)
    print(f"\nResults saved to: {results_file}")
    
    with open(RESULTS_PATH / 'meta_learning_summary.pkl', 'wb') as f:
        pickle.dump({
            'results': test_results,
            'meta_losses': meta_losses,
            'config': {
                'meta_lr': meta_lr,
                'inner_lr': inner_lr,
                'inner_steps': inner_steps,
                'meta_batch_size': meta_batch_size,
                'num_meta_iterations': num_meta_iterations,
                'k_shot_minority': k_shot_minority,
                'k_shot_majority': k_shot_majority,
                'q_query': q_query,
                'input_dim': input_dim,
                'hidden_dims': hidden_dims,
                'embedding_dim': embedding_dim
            },
            'timestamp': datetime.now().isoformat()
        }, f)
    print(f"Summary saved to: {RESULTS_PATH / 'meta_learning_summary.pkl'}")

else:
    print("\nNo test results generated")


META-LEARNING EVALUATION SUMMARY
   test_source  f1_macro  balanced_acc    auroc       mcc  test_samples  support_samples
  Zookeeper_2k  0.464143      0.634246 0.576143  0.284632          2000               15
      Linux_2k  0.457766      0.474141 0.466561 -0.026444          2000               15
        Mac_2k  0.379984      0.564228 0.545648  0.129553          2000               15
    Android_2k  0.338315      0.729990 0.892896  0.104651          2000               15
     Hadoop_2k  0.304979      0.317275 0.297044 -0.387549          2000               15
    Windows_2k  0.302047      0.488363 0.093377 -0.060682          2000               15
        HPC_2k  0.253060      0.312886 0.338485 -0.438630          2000               15
  Proxifier_2k  0.224257      0.278592 0.237639 -0.209639          2000               15
     Apache_2k  0.219481      0.476004 0.808558 -0.154828          2000               15
        BGL_2k  0.168943      0.225333 0.192585 -0.366068          2000     

In [14]:
print("\nPrototypical Network Evaluation...")

prototypical_results = []

model.eval()

proto_splits = splits[:3] if QUICK_TEST else splits

for split in proto_splits:
    test_source = split['test_source']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None or feat_variant not in test_data['feature_variants']:
        continue
    
    X_test = test_data['feature_variants'][feat_variant]
    y_test = test_data['labels']
    
    if len(np.unique(y_test)) < 2:
        continue
    
    scaler_test = StandardScaler()
    X_test_scaled = scaler_test.fit_transform(X_test)
    
    X_train_list = []
    y_train_list = []
    
    for src in split['train_sources']:
        if src in source_features:
            X_train_list.append(source_features[src])
            y_train_list.append(source_labels[src])
    
    if not X_train_list:
        continue
    
    X_train_combined = np.vstack(X_train_list)
    y_train_combined = np.concatenate(y_train_list)
    
    unique, counts = np.unique(y_train_combined, return_counts=True)
    minority_count = counts.min()
    k_proto = min(20, minority_count // 2)
    
    if k_proto < 5:
        continue
    
    episode = create_imbalanced_episode(
        X_train_combined, y_train_combined,
        k_proto, k_proto * 2, 0
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    with torch.no_grad():
        support_X_tensor = torch.FloatTensor(support_X).to(device)
        support_y_tensor = torch.LongTensor(support_y).to(device)
        support_embeddings = model(support_X_tensor)
        
        prototypes, proto_classes = compute_prototypes(support_embeddings, support_y_tensor)
        
        X_test_tensor = torch.FloatTensor(X_test_scaled).to(device)
        test_embeddings = model(X_test_tensor)
        
        distances = torch.cdist(test_embeddings, prototypes, p=2)
        test_preds = torch.argmin(distances, dim=1).cpu().numpy()
        
        probs = F.softmax(-distances, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test, test_preds, probs)
    
    prototypical_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'test_samples': len(y_test),
        'prototypes': len(prototypes)
    })
    
    print(f"\n{test_source} (Prototypical):")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")


Prototypical Network Evaluation...

Android_2k (Prototypical):
  F1-Macro: 0.3740
  Balanced Acc: 0.4613
  AUROC: 0.4873

Apache_2k (Prototypical):
  F1-Macro: 0.7000
  Balanced Acc: 0.7860
  AUROC: 0.9776

BGL_2k (Prototypical):
  F1-Macro: 0.4609
  Balanced Acc: 0.4979
  AUROC: 0.4848

Hadoop_2k (Prototypical):
  F1-Macro: 0.3248
  Balanced Acc: 0.3709
  AUROC: 0.2035

HealthApp_2k (Prototypical):
  F1-Macro: 0.2386
  Balanced Acc: 0.2023
  AUROC: 0.1738

HPC_2k (Prototypical):
  F1-Macro: 0.6822
  Balanced Acc: 0.6834
  AUROC: 0.5826

Linux_2k (Prototypical):
  F1-Macro: 0.2166
  Balanced Acc: 0.2532
  AUROC: 0.1819

Mac_2k (Prototypical):
  F1-Macro: 0.5104
  Balanced Acc: 0.5531
  AUROC: 0.5680

Proxifier_2k (Prototypical):
  F1-Macro: 0.4154
  Balanced Acc: 0.3734
  AUROC: 0.3074

Spark_2k (Prototypical):
  F1-Macro: 0.4762
  Balanced Acc: 0.9152
  AUROC: 0.9704

Thunderbird_2k (Prototypical):
  F1-Macro: 0.7134
  Balanced Acc: 0.7928
  AUROC: 0.8994

Windows_2k (Prototypical):


In [15]:
if prototypical_results:
    df_proto = pd.DataFrame(prototypical_results)
    df_proto = df_proto.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("PROTOTYPICAL NETWORK EVALUATION SUMMARY")
    print("="*80)
    print(df_proto.to_string(index=False))
    
    print("\n" + "="*60)
    print("PROTOTYPICAL AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(prototypical_results)}")
    print(f"Average F1-Macro: {df_proto['f1_macro'].mean():.4f} ± {df_proto['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_proto['balanced_acc'].mean():.4f} ± {df_proto['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_proto['auroc'].mean():.4f} ± {df_proto['auroc'].std():.4f}")
    
    proto_file = RESULTS_PATH / f"prototypical_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_proto.to_csv(proto_file, index=False)
    print(f"\nPrototypical results saved to: {proto_file}")


PROTOTYPICAL NETWORK EVALUATION SUMMARY
   test_source  f1_macro  balanced_acc    auroc       mcc  test_samples  prototypes
  Zookeeper_2k  0.875446      0.893592 0.951563  0.754650          2000           2
Thunderbird_2k  0.713395      0.792833 0.899428  0.458260          2000           2
     Apache_2k  0.699977      0.786024 0.977609  0.520686          2000           2
        HPC_2k  0.682177      0.683364 0.582615  0.365182          2000           2
        Mac_2k  0.510389      0.553081 0.567969  0.087924          2000           2
      Spark_2k  0.476247      0.915161 0.970444  0.138557          2000           2
        BGL_2k  0.460949      0.497882 0.484788 -0.002561          2000           2
  Proxifier_2k  0.415376      0.373358 0.307393 -0.127220          2000           2
    Android_2k  0.374027      0.461324 0.487316 -0.017738          2000           2
    Windows_2k  0.361068      0.359891 0.453685 -0.277294          2000           2
     Hadoop_2k  0.324830      0.370

In [16]:
print("\nFew-Shot Transfer Learning Evaluation...")

transfer_results = []

transfer_splits = splits[:3] if QUICK_TEST else splits

for split in transfer_splits:
    test_source = split['test_source']
    
    if test_source not in data_dict:
        continue
    
    test_data = data_dict[test_source]
    if test_data['labels'] is None or feat_variant not in test_data['feature_variants']:
        continue
    
    X_test_full = test_data['feature_variants'][feat_variant]
    y_test_full = test_data['labels']
    
    if len(np.unique(y_test_full)) < 2:
        continue
    
    scaler_transfer = StandardScaler()
    X_test_scaled = scaler_transfer.fit_transform(X_test_full)
    
    unique, counts = np.unique(y_test_full, return_counts=True)
    minority_count = counts.min()
    k_transfer = min(10, minority_count // 3)
    
    if k_transfer < 3:
        continue
    
    X_train_transfer, X_test_transfer, y_train_transfer, y_test_transfer = train_test_split(
        X_test_scaled, y_test_full, test_size=0.7, random_state=SEED, stratify=y_test_full
    )
    
    episode = create_imbalanced_episode(
        X_train_transfer, y_train_transfer,
        k_transfer, k_transfer * 2, 5
    )
    
    if episode[0] is None:
        continue
    
    support_X, support_y, _, _ = episode
    
    transfer_model = MetaLearner(input_dim, hidden_dims, embedding_dim, dropout, num_classes).to(device)
    transfer_model.load_state_dict(model.state_dict())
    
    transfer_optimizer = Adam(transfer_model.parameters(), lr=1e-3)
    
    support_X_tensor = torch.FloatTensor(support_X).to(device)
    support_y_tensor = torch.LongTensor(support_y).to(device)
    
    transfer_model.train()
    for epoch in range(50):
        transfer_optimizer.zero_grad()
        logits = transfer_model.predict(support_X_tensor)
        loss = focal_loss(logits, support_y_tensor)
        loss.backward()
        transfer_optimizer.step()
    
    transfer_model.eval()
    with torch.no_grad():
        X_test_tensor = torch.FloatTensor(X_test_transfer).to(device)
        test_logits = transfer_model.predict(X_test_tensor)
        test_probs = F.softmax(test_logits, dim=1).cpu().numpy()
        test_preds = torch.argmax(test_logits, dim=1).cpu().numpy()
    
    metrics = calculate_metrics(y_test_transfer, test_preds, test_probs)
    
    transfer_results.append({
        'test_source': test_source,
        'f1_macro': metrics['f1_macro'],
        'balanced_acc': metrics['balanced_acc'],
        'auroc': metrics['auroc'],
        'mcc': metrics['mcc'],
        'train_samples': len(support_y),
        'test_samples': len(y_test_transfer)
    })
    
    print(f"\n{test_source} (Transfer):")
    print(f"  F1-Macro: {metrics['f1_macro']:.4f}")
    print(f"  Balanced Acc: {metrics['balanced_acc']:.4f}")
    print(f"  AUROC: {metrics['auroc']:.4f}")
    
    del transfer_model
    torch.cuda.empty_cache()


Few-Shot Transfer Learning Evaluation...

Apache_2k (Transfer):
  F1-Macro: 0.9930
  Balanced Acc: 0.9960
  AUROC: 0.9996

BGL_2k (Transfer):
  F1-Macro: 0.9681
  Balanced Acc: 0.9753
  AUROC: 0.9964

Hadoop_2k (Transfer):
  F1-Macro: 0.9603
  Balanced Acc: 0.9673
  AUROC: 0.9905

HPC_2k (Transfer):
  F1-Macro: 0.9755
  Balanced Acc: 0.9772
  AUROC: 0.9983

Linux_2k (Transfer):
  F1-Macro: 0.9397
  Balanced Acc: 0.9936
  AUROC: 0.9997

Mac_2k (Transfer):
  F1-Macro: 0.9676
  Balanced Acc: 0.9703
  AUROC: 0.9844

Proxifier_2k (Transfer):
  F1-Macro: 1.0000
  Balanced Acc: 1.0000
  AUROC: 1.0000

Thunderbird_2k (Transfer):
  F1-Macro: 0.8563
  Balanced Acc: 0.9212
  AUROC: 0.9901

Windows_2k (Transfer):
  F1-Macro: 0.9829
  Balanced Acc: 0.9858
  AUROC: 0.9997

Zookeeper_2k (Transfer):
  F1-Macro: 0.9917
  Balanced Acc: 0.9957
  AUROC: 0.9999


In [17]:
if transfer_results:
    df_transfer = pd.DataFrame(transfer_results)
    df_transfer = df_transfer.sort_values('f1_macro', ascending=False)
    
    print("\n" + "="*80)
    print("TRANSFER LEARNING EVALUATION SUMMARY")
    print("="*80)
    print(df_transfer.to_string(index=False))
    
    print("\n" + "="*60)
    print("TRANSFER AGGREGATE STATISTICS")
    print("="*60)
    print(f"Sources evaluated: {len(transfer_results)}")
    print(f"Average F1-Macro: {df_transfer['f1_macro'].mean():.4f} ± {df_transfer['f1_macro'].std():.4f}")
    print(f"Average Balanced Acc: {df_transfer['balanced_acc'].mean():.4f} ± {df_transfer['balanced_acc'].std():.4f}")
    print(f"Average AUROC: {df_transfer['auroc'].mean():.4f} ± {df_transfer['auroc'].std():.4f}")
    
    transfer_file = RESULTS_PATH / f"transfer_results_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
    df_transfer.to_csv(transfer_file, index=False)
    print(f"\nTransfer results saved to: {transfer_file}")


TRANSFER LEARNING EVALUATION SUMMARY
   test_source  f1_macro  balanced_acc    auroc      mcc  train_samples  test_samples
  Proxifier_2k  1.000000      1.000000 1.000000 1.000000             30          1400
     Apache_2k  0.993041      0.996000 0.999580 0.986179             30          1400
  Zookeeper_2k  0.991654      0.995673 0.999904 0.983446             30          1400
    Windows_2k  0.982922      0.985784 0.999712 0.966235             30          1400
        HPC_2k  0.975496      0.977167 0.998339 0.951483             30          1400
        BGL_2k  0.968099      0.975303 0.996444 0.936355             30          1400
        Mac_2k  0.967551      0.970344 0.984365 0.935154             30          1400
     Hadoop_2k  0.960280      0.967293 0.990465 0.921677             30          1400
      Linux_2k  0.939747      0.993628 0.999693 0.886029             30          1400
Thunderbird_2k  0.856307      0.921235 0.990099 0.726888             30          1400

TRANSFER AGGREG

In [18]:
print("\n" + "="*80)
print("COMPARATIVE ANALYSIS")
print("="*80)

if test_results and prototypical_results and transfer_results:
    comparison_data = []
    
    sources_all = set([r['test_source'] for r in test_results])
    sources_proto = set([r['test_source'] for r in prototypical_results])
    sources_transfer = set([r['test_source'] for r in transfer_results])
    common_sources = sources_all & sources_proto & sources_transfer
    
    for source in common_sources:
        maml_result = next((r for r in test_results if r['test_source'] == source), None)
        proto_result = next((r for r in prototypical_results if r['test_source'] == source), None)
        transfer_result = next((r for r in transfer_results if r['test_source'] == source), None)
        
        if maml_result and proto_result and transfer_result:
            comparison_data.append({
                'Source': source,
                'MAML F1': maml_result['f1_macro'],
                'Proto F1': proto_result['f1_macro'],
                'Transfer F1': transfer_result['f1_macro'],
                'Best Method': max([
                    ('MAML', maml_result['f1_macro']),
                    ('Proto', proto_result['f1_macro']),
                    ('Transfer', transfer_result['f1_macro'])
                ], key=lambda x: x[1])[0]
            })
    
    if comparison_data:
        df_comparison = pd.DataFrame(comparison_data)
        print("\n" + df_comparison.to_string(index=False))
        
        print("\n" + "="*60)
        print("METHOD COMPARISON")
        print("="*60)
        print(f"MAML Average F1: {df_comparison['MAML F1'].mean():.4f}")
        print(f"Prototypical Average F1: {df_comparison['Proto F1'].mean():.4f}")
        print(f"Transfer Average F1: {df_comparison['Transfer F1'].mean():.4f}")
        
        best_counts = df_comparison['Best Method'].value_counts()
        print(f"\nBest method frequency:")
        for method, count in best_counts.items():
            print(f"  {method}: {count} times ({count/len(df_comparison)*100:.1f}%)")
        
        comparison_file = RESULTS_PATH / f"method_comparison_{datetime.now().strftime('%Y%m%d_%H%M%S')}.csv"
        df_comparison.to_csv(comparison_file, index=False)
        print(f"\nComparison saved to: {comparison_file}")

torch.save({
    'model': model.state_dict(),
    'config': {
        'input_dim': input_dim,
        'hidden_dims': hidden_dims,
        'embedding_dim': embedding_dim,
        'dropout': dropout,
        'num_classes': num_classes
    },
    'meta_config': {
        'meta_lr': meta_lr,
        'inner_lr': inner_lr,
        'inner_steps': inner_steps,
        'meta_batch_size': meta_batch_size,
        'num_meta_iterations': num_meta_iterations,
        'early_stopping_patience': early_stopping_patience,
        'curriculum_learning': True
    },
    'training_info': {
        'final_iteration': len(meta_losses),
        'best_meta_loss': best_meta_loss,
        'curriculum_phases': curriculum_phase + 1
    },
    'timestamp': datetime.now().isoformat()
}, MODELS_PATH / 'final_meta_model.pt')

print(f"\nFinal model saved to: {MODELS_PATH / 'final_meta_model.pt'}")


COMPARATIVE ANALYSIS

        Source  MAML F1  Proto F1  Transfer F1 Best Method
     Apache_2k 0.219481  0.699977     0.993041    Transfer
Thunderbird_2k 0.097269  0.713395     0.856307    Transfer
    Windows_2k 0.302047  0.361068     0.982922    Transfer
     Hadoop_2k 0.304979  0.324830     0.960280    Transfer
  Zookeeper_2k 0.464143  0.875446     0.991654    Transfer
      Linux_2k 0.457766  0.216627     0.939747    Transfer
        HPC_2k 0.253060  0.682177     0.975496    Transfer
        Mac_2k 0.379984  0.510389     0.967551    Transfer
        BGL_2k 0.168943  0.460949     0.968099    Transfer
  Proxifier_2k 0.224257  0.415376     1.000000    Transfer

METHOD COMPARISON
MAML Average F1: 0.2872
Prototypical Average F1: 0.5260
Transfer Average F1: 0.9635

Best method frequency:
  Transfer: 10 times (100.0%)

Comparison saved to: C:\Computer Science\AIMLDL\log-anomaly-detection\results\meta_learning\method_comparison_20251122_045037.csv

Final model saved to: C:\Computer Scien

In [19]:
try:
    import matplotlib.pyplot as plt
    
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    axes[0, 0].plot(meta_losses)
    axes[0, 0].set_title('Meta-Training Loss')
    axes[0, 0].set_xlabel('Iteration')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].grid(True, alpha=0.3)
    
    if test_results:
        sources = [r['test_source'] for r in test_results]
        f1_scores = [r['f1_macro'] for r in test_results]
        axes[0, 1].barh(sources, f1_scores)
        axes[0, 1].set_title('F1-Macro by Source (MAML)')
        axes[0, 1].set_xlabel('F1-Macro')
        axes[0, 1].grid(True, alpha=0.3)
    
    if comparison_data:
        methods = ['MAML', 'Proto', 'Transfer']
        avg_f1s = [
            df_comparison['MAML F1'].mean(),
            df_comparison['Proto F1'].mean(),
            df_comparison['Transfer F1'].mean()
        ]
        axes[1, 0].bar(methods, avg_f1s)
        axes[1, 0].set_title('Average F1-Macro by Method')
        axes[1, 0].set_ylabel('F1-Macro')
        axes[1, 0].grid(True, alpha=0.3)
    
    window = 50
    if len(meta_losses) > window:
        smoothed = np.convolve(meta_losses, np.ones(window)/window, mode='valid')
        axes[1, 1].plot(smoothed)
        axes[1, 1].set_title(f'Smoothed Meta-Training Loss (window={window})')
        axes[1, 1].set_xlabel('Iteration')
        axes[1, 1].set_ylabel('Loss')
        axes[1, 1].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(RESULTS_PATH / 'meta_learning_visualization.png', dpi=150, bbox_inches='tight')
    print(f"Visualization saved to: {RESULTS_PATH / 'meta_learning_visualization.png'}")
    plt.close()
except Exception as e:
    print(f"Visualization skipped: {e}")

Visualization saved to: C:\Computer Science\AIMLDL\log-anomaly-detection\results\meta_learning\meta_learning_visualization.png
